# Laboratorio 4: Aprendizaje Supervisado

> **Análisis de las Posiciones del Aborto en Twitter**
> *Diplomatura en Ciencias de Datos, Aprendizaje Automático y sus aplicaciones.*
> *Integrantes: 
    - Carlos Piaggio
    - Juan Pablo Analian
> Enunciado del TP: https://github.com/gonzigaran/AbortoTwitter/blob/master/lab4-AS.md    
> Nos enfocaremos en trabajar el texto. 
> Pasos realizados
    - Notebook 1: (este archivo) limpieza del Dataset de tweets de Junio 
    - Notebook 2: Trabajando con BoW en el texto, probamos los modelos vistos en el TP3
    - Notebook 3: Armamos 2 modelos de redes neuronales con Keras para comparar contra modelos anteriores
    
- [Notebook 2](entrega_04_v8.ipynb)
- [Notebook 3](entrega_04_v9.ipynb)

## Parte 1/3 (limpieza)    

In [2]:
import pandas as pd
import re
from nltk.corpus import stopwords

import seaborn as sns
import matplotlib.pyplot as plt

## 1. Cargamos datos originales
Levantamos el DS original de tweets de junio, ya que es el que mas registros tienen y ya están etiquetados.

In [2]:
df_full = pd.read_csv('data/tweets_junio.csv', parse_dates=['created_at'])

In [3]:
df_full.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1041585 entries, 0 to 1041584
Data columns (total 11 columns):
id                1041585 non-null int64
created_at        1041585 non-null datetime64[ns, UTC]
favorite_count    1041585 non-null int64
retweet_count     1041585 non-null int64
full_text         1041585 non-null object
posicion          1041585 non-null object
favorited         1041585 non-null bool
retweeted         1041585 non-null bool
user_name         1041529 non-null object
user_id           1041585 non-null int64
original_id       1041585 non-null int64
dtypes: bool(2), datetime64[ns, UTC](1), int64(5), object(3)
memory usage: 73.5+ MB


In [4]:
df_full.posicion.value_counts()

si    780455
no    261130
Name: posicion, dtype: int64

In [3]:
df_full.drop(inplace=True,columns=[ 'id','created_at','favorite_count','retweet_count','favorited','retweeted','user_name','user_id', 'original_id'])
df_full.columns = ['text','posicion']

In [5]:
display(df_full.head(5))

,text,posicion
0,RT @PTSarg: Contamos con una ventaja que ellos...,si
1,RT @juanabalmedina: Excelentes reflexiones de ...,si
2,En el fondo lo que les jode es nuestra liberta...,si
3,holla! me gustaría q ayuden a esta junta d fir...,si
4,RT @femialborto: A vos que decís #Salvemoslasd...,si


## 2. Limpiamos el texto

#### 2.1 Lista de Stopwords en español y hashtags

In [6]:
stop_words = list(stopwords.words('spanish'))
hashtags = pd.read_csv('./In/hashtags_clasificados_v2.csv')
lista = hashtags[ (hashtags.posicion == 1) | (hashtags.posicion == -1)]['hashtag'].str.replace('#','').tolist()
words_to_remove = stop_words + lista + ['rt']

print(lista[0:10])

['abortolegalya', 'abortolegal', 'quesealey', 'abortoseraley', 'salvemoslas2vidas', 'argentinaesprovida', 'noalabortoenargentina', 'abortosesionhistorica', 'queelabortosealey', 'noalaborto']


#### 2.2 Funcion de limpieza

In [7]:
REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|,;#¿?!¡,:]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z ñáéíóú]')
USERNAMES = re.compile(r"@[A-Za-z0-9]+")
STOPWORDS = set(words_to_remove)
HTTP_URL = re.compile(r"http\S+")
WWW_URL = re.compile(r"www\S+")

def clean_text(text):
 
    text = text.lower()  
    text = USERNAMES.sub('', text) # quitamos usernames (menciones)
    text = REPLACE_BY_SPACE_RE.sub(' ', text) # reemplazamos caracteres raros
    text = BAD_SYMBOLS_RE.sub('', text) # quitamos simbolos que NO estan en BAD_SYMBOLS_RE 
    text = HTTP_URL.sub('', text)  # quitamos url
    text = WWW_URL.sub('', text)  # quitamos url que comiencen con www
    
    text = text.replace('á', 'a')
    text = text.replace('é', 'e')
    text = text.replace('í', 'i')
    text = text.replace('ó', 'o')
    text = text.replace('ú', 'u')
    
    text = ' '.join(word for word in text.split() if word not in STOPWORDS) # remove stopwors from text
    return text

In [8]:
df_full['clean_text'] = df_full['text'].apply(clean_text)
display(df_full.head(10))

,text,posicion,clean_text
0,RT @PTSarg: Contamos con una ventaja que ellos...,si,contamos ventaja decision conviccion fuerza lucha
1,RT @juanabalmedina: Excelentes reflexiones de ...,si,excelentes reflexiones medica ginecologa catol...
2,En el fondo lo que les jode es nuestra liberta...,si,fondo jode libertad
3,holla! me gustaría q ayuden a esta junta d fir...,si,holla gustaria q ayuden junta d firmas seguro ...
4,RT @femialborto: A vos que decís #Salvemoslasd...,si,vos decis pido escuches intentes abrir mente e...
5,RT @slospennato: El debate de argumentos lo ga...,si,debate argumentos gano goleada legalizacion bi...
6,Porque somos dueñas de nuestrxs cuerpas #NoVot...,si,dueñas nuestrxs cuerpas
7,RT @CampAbortoLegal: Diputadxs: ¿Qué significa...,si,diputadxs significa ustedes mujer muerta credi
8,RT @CarrioJovenes: Que no coincidamos en el te...,si,coincidamos tema cambia respeto admiracion
9,#NoVotenContraLasMujeres estoy tan a la mitad ...,si,tan mitad tener postura definida tema entiendo...


#### Quitamos la columna temporal utilizada

In [9]:
df_full.drop(inplace=True,columns=['text'])
df_full = df_full.rename(columns={'clean_text': 'text'})
df_full.head(4)

,posicion,text
0,si,contamos ventaja decision conviccion fuerza lucha
1,si,excelentes reflexiones medica ginecologa catol...
2,si,fondo jode libertad
3,si,holla gustaria q ayuden junta d firmas seguro ...


#### Codificamos la etiqueta a predecir

Creamos un diccionario codificando la etiqueta a predecir.
donde el valor 'si', pasa a 1 (está a favor del aborto)
y el valor 'no', pasa a 0 (en contra del aborto)


In [10]:
codigo_posicion = {'no': 0, 'si': 1}

df_full['codigo_posicion'] = df_full['posicion']
df_full= df_full.replace({'codigo_posicion':codigo_posicion})

In [11]:
df_full.head(4)

,posicion,text,codigo_posicion
0,si,contamos ventaja decision conviccion fuerza lucha,1
1,si,excelentes reflexiones medica ginecologa catol...,1
2,si,fondo jode libertad,1
3,si,holla gustaria q ayuden junta d firmas seguro ...,1


In [12]:
df_full.drop(inplace=True,columns=['posicion'])
df_full.head(4)

,text,codigo_posicion
0,contamos ventaja decision conviccion fuerza lucha,1
1,excelentes reflexiones medica ginecologa catol...,1
2,fondo jode libertad,1
3,holla gustaria q ayuden junta d firmas seguro ...,1


In [20]:
df_full = df_full[df_full.text != '']

In [22]:
df_full.head(20)

,text,codigo_posicion
0,contamos ventaja decision conviccion fuerza lucha,1
1,excelentes reflexiones medica ginecologa catol...,1
2,fondo jode libertad,1
3,holla gustaria q ayuden junta d firmas seguro ...,1
4,vos decis pido escuches intentes abrir mente e...,1
5,debate argumentos gano goleada legalizacion bi...,1
6,dueñas nuestrxs cuerpas,1
7,diputadxs significa ustedes mujer muerta credi,1
8,coincidamos tema cambia respeto admiracion,1
9,tan mitad tener postura definida tema entiendo...,1


## 3. Salvamos el DF listo para trabajar en la siguiente notebook

In [ ]:
df_full.to_csv('./out/df_text_junio.csv', columns = ['codigo_posicion','text'], index=False)